In [2]:
!pip install gensim
!pip install pyLDAvis
!pip install wordcloud
!pip install matplotlib

import gensim
import pyLDAvis.gensim_models
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud


  Using cached gensim-4.3.1-cp38-cp38-win_amd64.whl (24.0 MB)
  Using cached smart_open-6.3.0-py3-none-any.whl (56 kB)
  Using cached scipy-1.10.1-cp38-cp38-win_amd64.whl (42.2 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.6.2
    Uninstalling scipy-1.6.2:
      Successfully uninstalled scipy-1.6.2
  Using cached pyLDAvis-3.4.0-py3-none-any.whl (2.6 MB)
  Using cached funcy-1.18-py2.py3-none-any.whl (33 kB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached scikit_learn-1.2.2-cp38-cp38-win_amd64.whl (8.3 MB)
  Using cached pandas-1.5.3-cp38-cp38-win_amd64.whl (11.0 MB)
  Using cached numpy-1.24.2-cp38-cp38-win_amd64.whl (14.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.1
    Uninstalling numpy-1.20.1:
      Successfully uninstalled numpy-1.20.1


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Esha\\anaconda3\\Lib\\site-packages\\~umpy\\core\\_multiarray_tests.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



  Using cached wordcloud-1.8.2.2-cp38-cp38-win_amd64.whl (152 kB)


AttributeError: module 'numpy' has no attribute '_no_nep50_warning'

In [1]:
import sys
# if you use python3 please change "pip" to "pip3"
!pip install numpy
!pip install pandas
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pprint import pprint

# Tutorial is from
#https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/
# read the above link for explanation

In [2]:
# Gensim
# !pip3 install gensim
!pip3 install spacy
!pip3 install matplot
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.utils import  simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
!pip install nltk
!python -m pip install nltk.downloader stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])


In [ ]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

In [ ]:
# Import Dataset
#The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics
df = pd.read_csv('./tweets/tweets.csv')
# df = df.loc[df.target_names.isin(['soc.religion.christian', 'rec.sport.hockey', 'talk.politics.mideast', 'rec.motorcycles']) , :]
print(df.shape)  #> (2361, 3)
df.head()

In [ ]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  

# Convert to list
data = df.text.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1])

from platform import python_version
print(python_version())

In [ ]:
#Lemmatisation in linguistics is the process of grouping together the inflected forms of a word so they can be analysed as a single item, identified by the word's lemma, or dictionary form.

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

import en_core_web_sm
nlp = en_core_web_sm.load()

#!python3 -m spacy download en_core_web_md 
# !python3 -m spacy download en  # run in terminal once
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
   # nlp = spacy.load('en', disable=['parser', 'ner'])
    nlp = spacy.load("en_core_web_sm")
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(data_words)  # processed Text Data!

In [ ]:
data_ready

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

# Build LDA model
model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

# pprint(model.print_topics())
# save model to disk (no need to use pickle module)
model.save('lda.model')



# print all topics
model.show_topics(2, 20)

# print topic 28
model.print_topic(3, topn=20)

# another way
for i in range(0, model.num_topics-1):
     print(model.print_topic(i))

# and another way, only prints top words
for t in range(0, model.num_topics-1):
    print(  'topic {}: '.format(t) + ', '.join([str(v[1]) for v in model.show_topic(t, 20)]))

logfile = open('../yourfile.txt', 'a')
#print>>logfile, lda.show_topics(topics=-1, topn=10) # for python2
print(logfile, model.show_topics(-1, 10))
for row in model.show_topics(-1,10):
    logfile.write(str(row)+'\n')    
    print(row)
logfile.close()

In [ ]:
!pip install gensim
!pip install pyLDAvis
!pip install wordcloud
!pip install matplotlib

import gensim
import pyLDAvis.gensim_models
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Visualize topics with pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
vis

# Plot WordCloud for each topic
for topic_id in range(num_topics):
    word_freqs = dict(lda_model.show_topic(topic_id, topn=30))
    wordcloud = WordCloud(width=800, height=800, background_color='white').generate_from_frequencies(word_freqs)
    plt.figure(figsize=(8, 8), facecolor=None)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.tight_layout(pad=0)
    plt.show()
